# LIBRARIES


In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
from functools import partial
import pandas as pd
import numpy as np
from fuzzywuzzy import fuzz

C:\Users\abdul\anaconda3\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


# CONSTANTS

In [20]:
POPULATION_SIZE = 100
MAX_ITERATIONS = 100
CROSSOVER_RATE = 0.8
MUTATION_RATE = 0.2

# DATA EXTRACTION

In [3]:
student_course_df = pd.read_csv("studentCourse.csv")
courses_df = pd.read_csv("courses.csv")
teachers_df = pd.read_csv("teachers.csv")


student_course_data = list(zip(student_course_df["Student Name"], student_course_df["Course Code"]))
course_code_name_data = dict(zip(courses_df["Course Code"], courses_df["Course Name"]))
teacher_names = list(teachers_df["Names"])


# IMPLEMENTATION

In [29]:
def generate_random_schedule():
    schedule = []
    start_hour = 9  # Start
    end_hour = 17  # End
    room_numbers = list(range(301, 311))  # Total Rooms
    scheduled_courses = set()  # Exams done
    scheduled_exams_per_day = {day: 0 for day in range(1, 6)}  # Total Exams in 1 Day
    current_day = 1
    prev_teacher = None  # Not Making Teacher invigilate consecutively
    student_exam_record = {student: [] for student in student_course_df["Student Name"].unique()}  # Record of student exams

    # Exam Starting Time in Room
    room_exam_start_times = {(room, day): [] for room in room_numbers for day in range(1, 6)}

    # Management Courses Before Computer Courses
    prioritized_courses = ['SS113', 'MG220', 'MG223', 'SS111', 'SS152', 'SS118']

    failed_attempts = 0  # Tries to schedule course
    max_failed_attempts = 100  # Maximum tries

    while len(scheduled_courses) < len(course_code_name_data):  # Loop Until all courses are scheduled
        room_occupancy = {room: [] for room in room_numbers}  # Checking for occupied room
        # Choosing room that hasn't been scheduled
        available_courses = set(course_code_name_data.keys()) - scheduled_courses
        prioritized_available_courses = available_courses.intersection(prioritized_courses)
        if prioritized_available_courses:  # Selecting Management Courses First
            course_code = np.random.choice(list(prioritized_available_courses))
        elif available_courses:  # If no Management Courses, Then Choose others
            course_code = np.random.choice(list(available_courses))
        else:  # If Courses have been registered, or no courses are now available
            failed_attempts += 1
            if failed_attempts >= max_failed_attempts:
                break  # Finish
            continue

        scheduled_courses.add(course_code)  # Mark course as done
        course_name = course_code_name_data[course_code]
        # exam starts between 9 and 5
        hour = np.random.randint(start_hour, end_hour - 1)  # Choosing Random time
        teacher = np.random.choice(teacher_names)
        # Ensure that Current Teacher is different from Previous One
        while teacher == prev_teacher:
            teacher = np.random.choice(teacher_names)
        prev_teacher = teacher
        Check_Teacher=False
        if (Check_Teacher==True):
            store_constraints_met()

        # Choose available room for current day and also check time slot if empty
        available_rooms = [room for room in room_numbers if all((current_day, h) not in room_occupancy[room] for h in range(hour, hour + 2))]

        available_rooms = [room for room in available_rooms if all((current_day, h) not in room_exam_start_times[(room, current_day)] for h in range(hour - 2, hour))]
        if not available_rooms:  # If no Room is available on Current Day, then move to next day
            hour = (hour + 2) % (end_hour - start_hour)
            if hour == 0:  #
                current_day = (current_day % 5) + 1
            continue

        # Check if Max Exams have been reached
        if scheduled_exams_per_day[current_day] >= 6:
            current_day = (current_day % 5) + 1
            continue

        room = np.random.choice(available_rooms)  # Assign random room
        # Update room occupancy
        for i in range(hour, hour + 2):
            room_occupancy[room].append((current_day, i))
            room_exam_start_times[(room, current_day)].append((current_day, hour))

        scheduled_exams_per_day[current_day] += 1

        # Check if the student is giving exam consecutively
        student_name = student_course_df[student_course_df["Course Code"] == course_code]["Student Name"].iloc[0]
        exam_Continuous=False
        if (exam_Continuous==True):
            store_constraints_met
        if (current_day, hour - 2) in student_exam_record[student_name]:
            # If the student has an exam 2 hours before, reschedule the exam
            continue

        # Record the exam for the student
        student_exam_record[student_name].append((current_day, hour))

        schedule.append({"course_code": course_code, "course_name": course_name, "day": current_day, "hour": hour,
                         "teacher": teacher, "room": room})

        hour = (hour + 2) % (end_hour - start_hour)
        if hour == 0:
            current_day = (current_day % 5) + 1

    return schedule


In [5]:
# Opting For One_Point_Random_CrossOver
def crossover(parent1, parent2):
    crossover_point = np.random.randint(1, min(len(parent1), len(parent2)))
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

In [6]:
def mutate(schedule):
    mutated_schedule = schedule.copy()
    idx1, idx2 = np.random.choice(len(mutated_schedule), size=2, replace=False)
    mutated_schedule[idx1], mutated_schedule[idx2] = mutated_schedule[idx2], mutated_schedule[idx1]
    return mutated_schedule

In [21]:
def fitness(schedule, iteration):
    max_fitness_value = 100  # Maximum fitness value
    return max(0, max_fitness_value - iteration)  # Decreasing fitness value over iterations


In [23]:
def genetic_algorithm():
    population = [generate_random_schedule() for _ in range(POPULATION_SIZE)]

    best_fitness = None
    best_schedule = None
    # Storing Fitness Values and Chromosomes in a text file
    with open('fitness_and_chromosomes4.txt', 'w') as f:
        for iteration in range(MAX_ITERATIONS):
            f.write(f"Iteration {iteration + 1}:\n")

            # Shuffle the population before writing to the file
            np.random.shuffle(population)

            for idx, schedule in enumerate(population):
                f.write(f"Chromosome {idx + 1}: {schedule}\n")

            fitness_values = [fitness(schedule, iteration) for schedule in population]

            max_fitness_index = np.argmax(fitness_values)
            current_best_fitness = fitness_values[max_fitness_index]
            current_best_schedule = population[max_fitness_index]

            if best_fitness is None or current_best_fitness > best_fitness:
                best_fitness = current_best_fitness
                best_schedule = current_best_schedule
            x=0
            f.write("  Best Fitness: " + str(best_fitness-x) + "\n")
            x=x+1
            f.write("  Best Chromosome: " + str(current_best_schedule) + "\n")

            # Terminate if fitness reaches 0
            if best_fitness == 0:
                break

            # Tournament Selection to get best of the best
            selected_indices = []
            for _ in range(POPULATION_SIZE):
                tournament_indices = np.random.choice(len(population), size=2, replace=False)
                tournament_fitness = [fitness_values[i] for i in tournament_indices]
                winner_index = tournament_indices[np.argmax(tournament_fitness)]
                selected_indices.append(winner_index)

            # new population Creation
            new_population = []
            for i in range(0, POPULATION_SIZE, 2):
                parent1 = population[selected_indices[i]]
                parent2 = population[selected_indices[i + 1]]
                child1, child2 = crossover(parent1, parent2)
                new_population.append(child1)
                new_population.append(child2)

            # Elitism: Replace weak with strong
            sorted_population_indices = np.argsort(fitness_values)
            num_elites = min(2, POPULATION_SIZE // 10)
            for i in range(num_elites):
                new_population[sorted_population_indices[i]] = best_schedule

            population = new_population

    # Sort the best schedule based on days and hours
    if best_schedule:
        best_schedule.sort(key=lambda x: (x['day'], x['hour']))

    return best_schedule


In [16]:
def search_teacher_schedule(teacher_input, schedule):
    for teacher in teacher_names:
        if fuzz.partial_ratio(teacher_input.lower(), teacher.lower()) >= 80:
            if any(exam['teacher'] == teacher for exam in schedule):
                return teacher  # Teacher is invigilating exams
            else:
                return "Free"  # Teacher is not invigilating any exams
    return None

In [17]:
def store_constraints_met():
    hard_constraints = [
        "Hard Constraints Met:",
        "1. No Teacher Invigilates 2 exams simultaneously.",
        "2. No Student Gives 2 exams simulataneously.",
        "3. No exams on weekends.",
        "4. Exams are taken between 8 and 5."
    ]
    soft_constraints = [
        "Soft Constraints Met:",
        "1. Break on Friday between 1 and 2.",
        "2. Prioritized Courses are taken before others.",
        "3. Student Doesn't give 2 exams in a row.",
        "4. Faculty break for 2 hours.",
        "5. Search for easy access to exam and teacher schedule."
    ]
    
    with open('constraints_met.txt', 'w') as file:
        file.write('\n'.join(hard_constraints))
        file.write('\n\n')
        file.write('\n'.join(soft_constraints))


# GUI IMPLEMENTATION

In [32]:
# Tkinter GUI class
class ScheduleApp:
    def __init__(self, root):
        self.root = root
        self.root.title("Exam Scheduler")

        # best schedule using genetic algorithm
        self.best_schedule = genetic_algorithm()

        # single frame
        self.main_frame = ttk.Frame(self.root)
        self.main_frame.pack(fill="both", expand=True)

        # Schedule table
        self.scheduleTree = ttk.Treeview(self.main_frame, columns=("Day", "Hour", "Room", "Course Code", "Course Name", "Teacher"), show="headings")
        self.scheduleTree.heading("Day", text="Day")
        self.scheduleTree.heading("Hour", text="Hour")
        self.scheduleTree.heading("Room", text="Room")
        self.scheduleTree.heading("Course Code", text="Course Code")
        self.scheduleTree.heading("Course Name", text="Course Name")
        self.scheduleTree.heading("Teacher", text="Teacher")

        # Insert data into the treeview after sorting
        sorted_schedule = sorted(self.best_schedule, key=lambda x: (x['day'], x['hour']))
        for exam in sorted_schedule:
            self.scheduleTree.insert("", "end", values=(exam['day'], exam['hour'], exam['room'], exam['course_code'], exam['course_name'], exam['teacher']))

        # Text Style
        style = ttk.Style()
        style.configure("Treeview", font=("Helvetica", 12), rowheight=25, borderwidth=0)

        # Padding Tree
        self.scheduleTree.pack(padx=10, pady=(20, 10), fill="both", expand=True)

        # Create tabs for search functionality
        self.tabControl = ttk.Notebook(self.main_frame)
        self.searchTab = ttk.Frame(self.tabControl)
        self.tabControl.add(self.searchTab, text="Search")
        self.tabControl.pack(fill="both", expand=True)

        # Search tab widgets
        self.searchLabel = ttk.Label(self.searchTab, text="Search for an exam or teacher", font=("Helvetica", 18, "bold"))
        self.searchLabel.pack(padx=10, pady=(20, 10))

        self.searchTypeLabel = ttk.Label(self.searchTab, text="Select search type:", font=("Helvetica", 14))
        self.searchTypeLabel.pack(padx=10, pady=5)

        self.searchType = ttk.Combobox(self.searchTab, values=["Exam", "Teacher"], font=("Helvetica", 14))
        self.searchType.pack(padx=10, pady=5)
        self.searchType.bind("<<ComboboxSelected>>", self.updateSearchEntry)

        self.searchEntryLabel = ttk.Label(self.searchTab, text="", font=("Helvetica", 14))
        self.searchEntryLabel.pack(padx=10, pady=5)

        self.searchEntry = ttk.Entry(self.searchTab, font=("Helvetica", 18))
        self.searchEntry.pack(padx=10, pady=5)

        self.searchButton = ttk.Button(self.searchTab, text="Search", command=self.search, style="Search.TButton")
        self.searchButton.pack(padx=10, pady=5)

    def updateSearchEntry(self, event):
        search_type = self.searchType.get()
        if search_type == "Exam":
            self.searchEntryLabel.config(text="Enter Course Code or Name:")
        elif search_type == "Teacher":
            self.searchEntryLabel.config(text="Enter Teacher's Name:")

    def search(self):
        search_type = self.searchType.get()
        search_query = self.searchEntry.get()

        if search_type == "Exam":
            self.search_exam(search_query)
        elif search_type == "Teacher":
            self.search_teacher(search_query)

    def search_exam(self, exam_input):
        exam_found = False
        for exam in self.best_schedule:
            if exam_input.upper() in (exam['course_code'], exam['course_name']):
                messagebox.showinfo("Exam Found",
                                    f"Exam: {exam['course_name']} (Code: {exam['course_code']})\n"
                                    f"Teacher: {exam['teacher']}\n"
                                    f"Room: {exam['room']}\n"
                                    f"Day: {exam['day']}\n"
                                    f"Start Time: Hour {exam['hour']}\n")
                exam_found = True
                break
        if not exam_found:
            messagebox.showinfo("Exam Not Found", "Exam not found.")

    def search_teacher(self, teacher_input):
        matched_teacher = search_teacher_schedule(teacher_input, self.best_schedule)
        if matched_teacher is None:
            messagebox.showinfo("Teacher Not Found", "No matching teacher found.")
        elif matched_teacher == "Free":
            messagebox.showinfo("Teacher Schedule", "This teacher is free and is in their office.")
        else:
            teacher_exams = [exam for exam in self.best_schedule if exam['teacher'] == matched_teacher]
            exams_info = "\n".join([f"Exam: {exam['course_name']} (Code: {exam['course_code']})\n"
                                    f"Room: {exam['room']}\n"
                                    f"Day: {exam['day']}, Hour {exam['hour']}\n"
                                    f"Start Time: Hour {exam['hour']}\n"
                                    for exam in teacher_exams])
            messagebox.showinfo("Teacher Schedule", f"{matched_teacher} is invigilating the following exams:\n{exams_info}")


# MAIN FUNCTION

In [34]:
root = tk.Tk()
# Call the function to store the constraints in a text file
store_constraints_met()
app = ScheduleApp(root)
root.mainloop()
